<a href="https://colab.research.google.com/github/vannis422/CIFAR-10/blob/main/CNN_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
from PIL import Image
import torch
import torchvision
import os
from sklearn.metrics import accuracy_score

class SimpleCNN(torch.nn.Module):
    def __init__(self, num_classes=10, fc_hidden_size=256, dropout_ratio=0.5):
        super().__init__()

        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2) # 2x2 pooling

        self.fc1 = torch.nn.Linear(128 * 4 * 4, fc_hidden_size)
        self.fc2 = torch.nn.Linear(fc_hidden_size, num_classes)
        self.relu = torch.nn.ReLU() # 非線性轉換

        self.dropout = torch.nn.Dropout(dropout_ratio)
    def forward(self, x):
        # ✅ conv1 layer + ReLU + pooling
        x = self.relu(self.conv1(x))     # conv1 → [B, 3, 32, 32] → [B, 32, 32, 32]
        x = self.pool(x)                 # pool → [B, 32, 16, 16]

        # ✅ conv2 layer + ReLU + pooling
        x = self.relu(self.conv2(x))     # conv2 → [B, 64, 16, 16]
        x = self.pool(x)                 # pool → [B, 64, 8, 8]

        # ✅ conv3 layer + ReLU + pooling
        x = self.relu(self.conv3(x))     # conv3 → [B, 128, 8, 8]
        x = self.pool(x)                 # pool → [B, 128, 4, 4]

        # ✅ 展平成向量
        x = x.view(x.size(0), -1)        # → [B, 128*4*4 = 2048]

        # ✅ 全連接層 + ReLU + Dropout
        x = self.relu(self.fc1(x))       # fc1 → [B, 256]
        x = self.dropout(x)

        # ✅ 最後一層輸出分類
        x = self.fc2(x)                  # fc2 → [B, 10]
        return x